In [ ]:
%pip install textblob


In [ ]:
# import libraries
from textblob import TextBlob
import pandas as pd
import time
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
    accuracy_score
)
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../../../data/test_data.csv")  # Load test dataset

print(df.head())

In [ ]:
# sentiment analysis
def get_textblob_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity
    return pd.Series([polarity, subjectivity])

df[['Polarity_TextBlob', 'Subjectivity_TextBlob']] = df['text'].apply(get_textblob_sentiment)


In [ ]:
# Convert polarity to class
def polarity_to_class(polarity):
    if polarity > 0:
        return 'Positive'
    elif polarity < 0:
        return 'Negative'
    else:
        return 'Neutral'

In [ ]:
# Initialize time for recording inference time
start = time.time()

df['TextBlob_Prediction'] = df['Polarity_TextBlob'].apply(polarity_to_class) # Apply TextBlob sentiment analysis

end = time.time()
inference_time = end - start

# Save computation time to CSV
time_df = pd.DataFrame({"TextBlob_InferenceTime (s)": [inference_time]})
time_df.to_csv("computation_time_textblob.csv", index=False)

print("Inference time: ", inference_time, "seconds")
print("Inference time is saved")

Save result to CSV

In [ ]:
# Select only desired columns
result_df = df[["id", "text", "TextBlob_Sentiment"]]

# Save to CSV
result_df.to_csv("textblob_sentiment_output.csv", index=False)

print("Sentiment analysis complete. Output saved to 'textblob_sentiment_output.csv'.")

print("Output length:", len(result_df))

Evaluation

In [ ]:
# Load the results
df_vader = pd.read_csv("textblob_sentiment_output.csv")

y_true = df["Analysis"] # True value
y_pred = df_vader["TextBlob_Sentiment"] # Predicted value

Classification Report

In [ ]:
report = classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()

In [ ]:
# Plot per-class Precision, Recall, F1-score
report_df.drop(['accuracy', 'macro avg', 'weighted avg'], inplace=True)
report_df[['precision', 'recall', 'f1-score']].plot.bar(figsize=(10,6))
report_df[['precision', 'recall', 'f1-score']].to_csv("classification_report_textblob.csv") # Save classification report to CSV

plt.title("Per-Class Precision, Recall, F1-score")
plt.ylabel("Score")
plt.xlabel("Sentiment Class")
plt.ylim(0, 1)
plt.grid(True)
plt.tight_layout()
plt.savefig("classification_report_textblob.png") # Save classification report plot
plt.show()

Confusion Matrix

In [ ]:
cm = confusion_matrix(y_true, y_pred, labels=report_df.index)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=report_df.index)
disp.plot(cmap=plt.cm.Blues)

plt.title("Confusion Matrix")
plt.tight_layout()
plt.savefig("confusion_matrix_textblob.png") # Save confusion matrix plot
plt.show()

Accuracy

In [ ]:
acc = accuracy_score(y_true, y_pred)
rounded_acc = round(acc, 4) # Round accuracy to 4 decimal places
print("Overall Accuracy:", rounded_acc)

accuracy_df = pd.DataFrame([{"Metric": "Overall Accuracy", "Score": rounded_acc}])
accuracy_df.to_csv("overall_accuracy_textblob.csv", index=False) # Save overall accuracy to CSV
